# Customer.IO Profile Aliasing

This notebook demonstrates how to use profile aliasing in Customer.IO to link user identities.
Aliasing is essential for connecting anonymous users to identified users when they sign up or log in.

## Setup and Imports

Import the alias management functions and initialize the API client.

In [ ]:
# Setup and imports
import os
from datetime import datetime
from utils.api_client import CustomerIOClient
from utils.people_manager import identify_user
from utils.event_manager import track_event
from utils.alias_manager import create_alias, merge_profiles
from utils.exceptions import CustomerIOError, ValidationError

# Initialize client
API_KEY = os.getenv('CUSTOMERIO_API_KEY', 'your-api-key-here')
client = CustomerIOClient(api_key=API_KEY, region='us')

print("Customer.IO profile aliasing functions loaded successfully")

## Understanding Profile Aliasing

Profile aliasing in Customer.IO:
- **Links** different user identifiers to the same person
- **Merges** anonymous activity with identified user profiles
- **Consolidates** user data from multiple sources or sessions
- **Maintains** complete user journey history

Common scenarios:
- Anonymous user signs up or logs in
- User logs in with different email/username
- Merging data from multiple systems
- Consolidating cross-device activity

## Basic Aliasing Operations

Create aliases to link user identities.

In [ ]:
# Example: Anonymous user signs up
anonymous_id = "anon_abc123"
user_id = "user_signed_up_001"

# First, identify the user to ensure they exist
try:
    identify_user(
        client=client,
        user_id=user_id,
        traits={"email": f"{user_id}@example.com", "name": "Signed Up User"}
    )
    print(f"User {user_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying user: {e}")

try:
    result = create_alias(
        client=client,
        user_id=user_id,
        previous_id=anonymous_id
    )
    print(f"Successfully linked anonymous user {anonymous_id} to {user_id}")
except CustomerIOError as e:
    print(f"Error creating alias: {e}")

In [ ]:
# Example: User logs in with different identifier
email_id = "user@example.com"
database_id = "db_user_12345"

# First, identify the user to ensure they exist
try:
    identify_user(
        client=client,
        user_id=database_id,
        traits={"email": email_id, "name": "Database User"}
    )
    print(f"User {database_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying user: {e}")

try:
    result = create_alias(
        client=client,
        user_id=database_id,
        previous_id=email_id,
        timestamp=datetime.now()
    )
    print(f"Linked email {email_id} to database ID {database_id}")
except CustomerIOError as e:
    print(f"Error: {e}")

## Real-World Example: User Registration Flow

Complete workflow for handling user registration and identity consolidation.

In [ ]:
class UserRegistrationFlow:
    """Handle user registration with proper aliasing."""
    
    def __init__(self, client):
        self.client = client
    
    def handle_signup(self, anonymous_id, user_email, user_traits):
        """Process user signup and create proper aliases."""
        
        # Step 1: Create alias linking anonymous to email
        try:
            create_alias(
                self.client,
                user_id=user_email,
                previous_id=anonymous_id
            )
            print(f"SUCCESS: Linked anonymous {anonymous_id[:8]}... to {user_email}")
        except CustomerIOError as e:
            print(f"ERROR: Alias creation failed: {e}")
            return False
        
        # Step 2: Update user profile with registration data
        try:
            identify_user(
                client=self.client,
                user_id=user_email,
                traits={
                    **user_traits,
                    "registration_date": datetime.now().isoformat(),
                    "user_status": "registered"
                }
            )
            print(f"SUCCESS: Updated profile for {user_email}")
        except CustomerIOError as e:
            print(f"ERROR: Profile update failed: {e}")
            return False
        
        # Step 3: Track registration event
        try:
            track_event(
                client=self.client,
                user_id=user_email,
                event="User Registered",
                properties={
                    "registration_method": "email_signup",
                    "had_anonymous_activity": True,
                    "previous_anonymous_id": anonymous_id
                }
            )
            print(f"SUCCESS: Tracked registration event for {user_email}")
            return True
        except CustomerIOError as e:
            print(f"ERROR: Event tracking failed: {e}")
            return False

In [ ]:
# Example: Process user registration
registration_flow = UserRegistrationFlow(client)

# Simulate anonymous user becoming registered
anonymous_visitor = "anon_visitor_xyz789"
new_user_email = "newuser@example.com"
user_data = {
    "email": new_user_email,
    "name": "Jane Smith",
    "plan": "free",
    "source": "organic_search"
}

success = registration_flow.handle_signup(
    anonymous_visitor, 
    new_user_email, 
    user_data
)

if success:
    print("\nSUCCESS: User registration completed successfully!")
else:
    print("\nERROR: User registration failed.")

## Profile Merging

Merge profiles when users have multiple identities that need to be consolidated.

In [ ]:
# Example: Merge user profiles
primary_user_id = "user_primary_001"
secondary_user_id = "user_secondary_001"

# First, ensure both users exist
try:
    identify_user(
        client=client,
        user_id=primary_user_id,
        traits={"email": f"{primary_user_id}@example.com", "name": "Primary User"}
    )
    print(f"Primary user {primary_user_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying primary user: {e}")

try:
    identify_user(
        client=client,
        user_id=secondary_user_id,
        traits={"email": f"{secondary_user_id}@example.com", "name": "Secondary User"}
    )
    print(f"Secondary user {secondary_user_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying secondary user: {e}")

try:
    result = merge_profiles(
        client=client,
        primary_user_id=primary_user_id,
        secondary_user_id=secondary_user_id,
        merge_reason="duplicate_detection",
        merge_source="data_cleanup"
    )
    print(f"Merged {secondary_user_id} into {primary_user_id}")
except CustomerIOError as e:
    print(f"Error merging profiles: {e}")

## Cross-Platform Identity Management

Handle users across multiple platforms and devices.

In [ ]:
def consolidate_cross_platform_user(client, identities):
    """Consolidate user across multiple platforms."""
    
    # Use email as primary identifier
    primary_id = identities["email"]
    
    # Create aliases for all other identifiers
    for platform, identifier in identities.items():
        if platform != "email" and identifier:
            try:
                create_alias(
                    client,
                    user_id=primary_id,
                    previous_id=identifier
                )
                print(f"SUCCESS: Linked {platform} ID: {identifier}")
            except CustomerIOError as e:
                print(f"ERROR: Failed to link {platform}: {e}")
    
    return primary_id

# Example: User with multiple platform identities
user_identities = {
    "email": "multiplatform@example.com",
    "mobile_app": "mobile_user_abc",
    "web_anonymous": "web_anon_123",
    "social_login": "google_oauth_xyz",
    "legacy_system": "legacy_id_456"
}

primary_identity = consolidate_cross_platform_user(client, user_identities)
print(f"\nPrimary identity: {primary_identity}")

## Advanced Aliasing Scenarios

Handle complex identity management situations.

In [ ]:
# Example: Email change handling
def handle_email_change(client, user_id, old_email, new_email):
    """Handle user email address changes."""
    
    # First ensure user exists
    try:
        identify_user(
            client=client,
            user_id=user_id,
            traits={"email": old_email, "name": "User Before Email Change"}
        )
        print(f"User {user_id} identified with old email")
    except CustomerIOError as e:
        print(f"Error identifying user: {e}")
        return False
    
    try:
        # Create alias from new email to current user ID
        create_alias(
            client,
            user_id=user_id,
            previous_id=new_email
        )
        
        # Update user profile with new email
        identify_user(
            client=client,
            user_id=user_id,
            traits={
                "email": new_email,
                "previous_email": old_email,
                "email_changed_at": datetime.now().isoformat()
            }
        )
        
        # Track email change event
        track_event(
            client=client,
            user_id=user_id,
            event="Email Changed",
            properties={
                "old_email": old_email,
                "new_email": new_email
            }
        )
        
        print(f"SUCCESS: Email updated from {old_email} to {new_email}")
        return True
        
    except CustomerIOError as e:
        print(f"ERROR: Email change failed: {e}")
        return False

# Example usage
handle_email_change(
    client,
    "user_email_change_001",
    "oldname@example.com",
    "newname@example.com"
)

In [ ]:
# Example: Account migration between systems
def migrate_user_account(client, old_system_id, new_system_id, migration_data):
    """Migrate user from old system to new system."""
    
    try:
        # Create alias linking old to new system
        create_alias(
            client,
            user_id=new_system_id,
            previous_id=old_system_id,
            context={
                "migration": {
                    "source_system": migration_data["source"],
                    "target_system": migration_data["target"],
                    "migration_date": datetime.now().isoformat()
                }
            }
        )
        
        # Update profile with migration info
        identify_user(
            client=client,
            user_id=new_system_id,
            traits={
                **migration_data["user_traits"],
                "migrated_from": old_system_id,
                "migration_completed": True
            }
        )
        
        # Track migration event
        track_event(
            client=client,
            user_id=new_system_id,
            event="Account Migrated",
            properties={
                "from_system": migration_data["source"],
                "to_system": migration_data["target"],
                "data_preserved": True
            }
        )
        
        print(f"SUCCESS: Migrated {old_system_id} to {new_system_id}")
        return True
        
    except CustomerIOError as e:
        print(f"ERROR: Migration failed: {e}")
        return False

# Example migration
migration_info = {
    "source": "legacy_crm",
    "target": "new_platform",
    "user_traits": {
        "email": "migrated@example.com",
        "name": "Migrated User",
        "account_type": "premium"
    }
}

migrate_user_account(
    client,
    "legacy_user_789",
    "new_platform_user_789",
    migration_info
)

## Error Handling and Validation

Handle aliasing errors and validation issues.

In [ ]:
# Example: Common aliasing errors
invalid_alias_operations = [
    # Empty user ID
    lambda: create_alias(client, "", "previous_123"),
    # Same user ID and previous ID
    lambda: create_alias(client, "user_123", "user_123"),
    # Empty previous ID
    lambda: create_alias(client, "user_123", ""),
    # Invalid context type
    lambda: create_alias(client, "user_123", "prev_123", context="not a dict")
]

for i, operation in enumerate(invalid_alias_operations):
    try:
        operation()
    except ValidationError as e:
        print(f"Validation error {i+1}: {e}")
    except CustomerIOError as e:
        print(f"API error {i+1}: {e}")

## Best Practices

### Identity Strategy
- **Use email as primary identifier** when possible
- **Create aliases immediately** when users sign up or log in
- **Track identity changes** with events for audit trails
- **Handle anonymous-to-identified transitions** smoothly

### Data Integrity
- **Validate identifiers** before creating aliases
- **Avoid circular aliases** (A->B->A)
- **Document identity mapping** strategy for your team
- **Test identity flows** thoroughly

### Performance
- **Batch alias operations** when possible
- **Cache identity mappings** to reduce API calls
- **Use consistent identifier formats** across systems
- **Monitor alias success rates** for troubleshooting

### Privacy and Compliance
- **Handle PII carefully** in alias operations
- **Support identity deletion** for GDPR compliance
- **Log identity changes** for audit requirements
- **Secure identity data** in transit and storage

## Next Steps

Now that you understand profile aliasing, explore:

- **01_people_management.ipynb** - Manage user profiles and identification
- **02_event_tracking.ipynb** - Track events for aliased users
- **05_batch_operations.ipynb** - Bulk alias operations for data migrations